# 完成自己的 toy_git

翻译自大神的文章：https://benhoyt.com/writings/pygit/

最后实现的 toy_git 参考大神的仓库：https://github.com/benhoyt/pygit

## 初始化一个仓库

初始化本地 Git 仓库只需要创建 .git 目录及其下的一些文件和目录。在定义了 read_file 和 write_file 辅助函数之后，我们就可以编写 init()

In [1]:
def init(repo):
    """Create directory for repo and initialize .git directory."""
    os.mkdir(repo)
    os.mkdir(os.path.join(repo, '.git'))
    for name in ['objects', 'refs', 'refs/heads']:
        os.mkdir(os.path.join(repo, '.git', name))
    write_file(os.path.join(repo, '.git', 'HEAD'),
               b'ref: refs/heads/master')
    print('initialized empty repository: {}'.format(repo))


你会注意到，没有很多优雅的错误处理。毕竟，这是一个 500 行的项目。如果 repo 目录已经存在，那么使用回溯就会失败。

## 散列对象

hash_object() 将对象散列并写入 `.git/objects`（“数据库”）。 Git 模型中有三种类型的对象：blob（普通文件），提交（commit）和树（tree）（这些表示单个目录的状态）。 每个对象都有一个小 header，包括字节的类型和大小。接下来是 NUL 字节，然后是文件的数据字节。所有文件都被 zlib 压缩并写入`.git/objects/ab/cd...`，其中 ab 是 40 个字符的 SHA-1 hash 的前两个字符，而 cd... 是剩余的字符的。 请注意使用Python 标准库（ os 和 hashlib）。

In [3]:
def hash_object(data, obj_type, write=True):
    """Compute hash of object data of given type and write to object store
    if "write" is True. Return SHA-1 object hash as hex string.
    """
    header = '{} {}'.format(obj_type, len(data)).encode()
    full_data = header + b'\x00' + data
    sha1 = hashlib.sha1(full_data).hexdigest()
    if write:
        path = os.path.join('.git', 'objects', sha1[:2], sha1[2:])
        if not os.path.exists(path):
            os.makedirs(os.path.dirname(path), exist_ok=True)
            write_file(path, zlib.compress(full_data))
    return sha1

然后是 find_object()，它通过散列（或散列前缀）找到一个对象。read_object() 读取一个对象及其类型 - 基本上是 hash_object() 的逆。最后，cat_file 是一个实现 git cat-file 的 pygit 等价的函数：它将对象的内容（或其大小或类型）打印到 stdout。

## Git 索引

我们接下来想做的事情就是将文件添加到索引或暂存区域中。索引是文件的入口，按照 path 排序，每个索引包含 path，修改时间，SHA-1 hash 等等。索引只包含上一次 push 之后到这次 add 之后的所有文件。

索引是单文件在 `.git/index` 以自定义的二进制保存。并不是很复杂，但是为了得到可变长度路径之后的下一个索引条目确实需要一些 struct 上的用法以及加上一点点骚操作：

前 12 个字节是 header，最后 20 个字节是前面所有字节的 SHA-1 hash，中间的字节是索引的入口。这些索引是由 62 个字节以及加上路径的长度和一些填充值。

这里是 IndexEntry namedtuple 和 read_index()

In [1]:
import collections
# Data for one entry in the git index (.git/index)
IndexEntry = collections.namedtuple('IndexEntry', [
    'ctime_s', 'ctime_n', 'mtime_s', 'mtime_n', 'dev', 'ino', 'mode',
    'uid', 'gid', 'size', 'sha1', 'flags', 'path',
])

def read_index():
    """Read git index file and return list of IndexEntry objects."""
    try:
        data = read_file(os.path.join('.git', 'index'))
    except FileNotFoundError:
        return []
    digest = hashlib.sha1(data[:-20]).digest()
    assert digest == data[-20:], 'invalid index checksum'
    signature, version, num_entries = struct.unpack('!4sLL', data[:12])
    assert signature == b'DIRC', \
            'invalid index signature {}'.format(signature)
    assert version == 2, 'unknown index version {}'.format(version)
    entry_data = data[12:-20]
    entries = []
    i = 0
    while i + 62 < len(entry_data):
        fields_end = i + 62
        fields = struct.unpack('!LLLLLLLLLL20sH',
                               entry_data[i:fields_end])
        path_end = entry_data.index(b'\x00', fields_end)
        path = entry_data[fields_end:path_end]
        entry = IndexEntry(*(fields + (path.decode(),)))
        entries.append(entry)
        entry_len = ((62 + len(path) + 8) // 8) * 8
        i += entry_len
    assert len(entries) == num_entries
    return entries

此函数后跟 ls_files，status 和 diff 函数，所有这些函数实质上都是打印索引状态的不同方法

+ ls_file 只是打印在索引中的所有文件（如果 if -s 被指定的话还有它们的 mode 和 hash）


+ status 使用 get_status() 将索引中的文件与当前目录树中的文件进行比较，并打印出哪些文件被修改，建立和删除


+ diff 打印每个修改过的文件的差异，显示索引中的内容与当前工作副本中的内容（使用 Python 的 difflib）

考虑到文件修改时间和所有这些，我 100％ 肯定 git 对索引的使用和这些命令的实现比我的更有效。我只是通过 os.walk() 来列出完整目录列表，并使用一些集合的操作然后比较哈希。例如，这是我用来确定更改路径列表的集合理解


```python

changed = {p for p in (paths & entry_paths)
           if hash_object(read_file(p), 'blob', write=False) !=
              entries_by_path[p].sha1.hex()}
```

最后有一个 write_index 函数用于将索引写回，而 add() 用于向索引添加一个或多个路径 - 后者只是读取整个索引，添加路径，重新排序并再次写出来。

此时我们可以将文件添加到索引中，我们已准备好进行 commit。

## Commit

Commit 包括编写两个对象

首先，树对象，它是提交时当前目录（或实际上是索引）的快照。树只列出目录中文件（blob）和子树的哈希值 - 它是递归的。

所以每一个提交都是整个目录树的快照。但是通过 hash 保存数据的美妙之处在于如果在树中有任何文件发生了改变，整个树的 hash 也要发生改变。反过来，如果一个文件或者子树没有改变，hash 不会发生改变。所以你可以高效地存储树目录的变化。

这是一个由 cat-file pretty 2226 打印的树对象的示例（每行显示文件模式，对象类型，哈希和文件名）


    100644 blob 4aab5f560862b45d7a9f1370b1c163b74484a24d    LICENSE.txt
    100644 blob 43ab992ed09fa756c56ff162d5fe303003b5ae0f    README.md
    100644 blob c10cb8bc2c114aba5a1cb20dea4c1597e5a3c193    pygit.py
    
write_tree() 这个函数非常奇怪，用来写树对象。在 Git 里面有一个奇怪的事情，有些文件混合着二进制和文本。比如说，在树对象中的每一行，是 “mode space path” 的文本。接下来是 NUL byte，接下来是二进制的 SHA-1 hash。 这里是我们的 write_tree() 

In [1]:
def write_tree():
    """Write a tree object from the current index entries."""
    tree_entries = []
    for entry in read_index():
        assert '/' not in entry.path, \
                'currently only supports a single, top-level directory'
        mode_path = '{:o} {}'.format(entry.mode, entry.path).encode()
        tree_entry = mode_path + b'\x00' + entry.sha1
        tree_entries.append(tree_entry)
    return hash_object(b''.join(tree_entries), 'tree')

接着是 commit 对象，这个记录着 tree hash，父提交，作者和时间戳以及提交的文本信息。合并当然是关于 Git 的好东西之一，但是我们的 toy_git 只支持单个线性分支，所以只有一个父级（或者在第一次提交的情况下没有父级！）。

这是一个提交对象的例子，再次使用 cat-file pretty aa8d 打印出来：

    tree 22264ec0ce9da29d0c420e46627fa0cf057e709a
    parent 03f882ade69ad898aba73664740641d909883cdc
    author Ben Hoyt <benhoyt@gmail.com> 1493170892 -0500
    committer Ben Hoyt <benhoyt@gmail.com> 1493170892 -0500
    


这是我们的提交函数

In [2]:
def commit(message, author):
    """Commit the current state of the index to master with given message.
    Return hash of commit object.
    """
    tree = write_tree()
    parent = get_local_master_hash()
    timestamp = int(time.mktime(time.localtime()))
    utc_offset = -time.timezone
    author_time = '{} {}{:02}{:02}'.format(
            timestamp,
            '+' if utc_offset > 0 else '-',
            abs(utc_offset) // 3600,
            (abs(utc_offset) // 60) % 60)
    lines = ['tree ' + tree]
    if parent:
        lines.append('parent ' + parent)
    lines.append('author {} {}'.format(author, author_time))
    lines.append('committer {} {}'.format(author, author_time))
    lines.append('')
    lines.append(message)
    lines.append('')
    data = '\n'.join(lines).encode()
    sha1 = hash_object(data, 'commit')
    master_path = os.path.join('.git', 'refs', 'heads', 'master')
    write_file(master_path, (sha1 + '\n').encode())
    print('committed to master: {:7}'.format(sha1))
    return sha1

## 与服务器交互

接下来是更难的部分，其中我们将 toy_git 与一个真实的 Git 服务器进行交互（我将 toy_git 推送到 GitHub，但它也适用于 Bitbucket 和其他服务器）。

基本思想是查询服务器的主分支以知道我们所提交的内容，然后决定哪些对象的集合需要加入当前提交。最后更新远程仓库的 commit hash 然后向它提交缺失对象的 “pack file”。

这被叫做 “smart protocal”—— 2011 年开始。GitHub 停止对 “dumb” 的传输协议的支持，它只是简单地传输 .git 文件，并且实现起来会更容易一些。

不幸的是，当我在实现 “smart protocol” 我犯了一个愚蠢的错误——直到实现，我都没有在 HTTP protocal 和 pack protocol 找到关键的技术文档。

在使其工作的最后阶段，我还使用 Python 的 http.server 模块实现了一个小型 HTTP 服务器，因此我可以针对它运行常规 git 客户端并查看一些实际请求。

### pkt-line 格式

传输协议的关键部分之一是所谓的“pkt-line”格式，它是一种长度前缀的数据包格式，用于发送像提交哈希这样的元数据。每“行”具有 4 位十六进制长度（加上 4 位去包括该行长度的长度），然后长度少于 4 个字节的数据。每行通常在末尾还有一个 LF 字节。特殊长度 0000 用作节标记并位于数据的末尾。

例如，这是 GitHub 对 git-receive-pack GET 请求的响应。请注意，其他换行符和缩进不是实际数据的一部分：

```
    001f# service=git-receive-pack\n
    0000
    00b20000000000000000000000000000000000000000 capabilities^{}\x00
        report-status delete-refs side-band-64k quiet atomic ofs-delta
        agent=git/2.9.3~peff-merge-upstream-2-9-1788-gef730f7\n
    0000
```

所以我们需要两个函数，一个用于将 pkt-line 数据转换为一个行列表，另一个用于将行列表转换为 pkt-line 格式：

In [3]:
def extract_lines(data):
    """Extract list of lines from given server data."""
    lines = []
    i = 0
    for _ in range(1000):
        line_length = int(data[i:i + 4], 16)
        line = data[i + 4:i + line_length]
        lines.append(line)
        if line_length == 0:
            i += 4
        else:
            i += line_length
        if i >= len(data):
            break
    return lines

def build_lines_data(lines):
    """Build byte string from given lines to send to server."""
    result = []
    for line in lines:
        result.append('{:04x}'.format(len(line) + 5).encode())
        result.append(line)
        result.append(b'\n')
    result.append(b'0000')
    return b''.join(result)

### 发 https 请求

下一个技巧 - 因为我只想使用标准库 - 在没有 requests 库的情况下进行经过身份验证的 HTTPS 请求。这是代码：

In [4]:
def http_request(url, username, password, data=None):
    """Make an authenticated HTTP request to given URL (GET by default,
    POST if "data" is not None).
    """
    password_manager = urllib.request.HTTPPasswordMgrWithDefaultRealm()
    password_manager.add_password(None, url, username, password)
    auth_handler = urllib.request.HTTPBasicAuthHandler(password_manager)
    opener = urllib.request.build_opener(auth_handler)
    f = opener.open(url, data=data)
    return f.read()

这就是 requests 为什么要存在的原因。你可以使用标准库的 urllib.request 模块执行所有操作，但有时会很痛苦。大多数 Python stdlib 都很棒。使用 requests 的等效代码实际上甚至不需要辅助函数：

In [5]:
def http_request(url, username, password):
    response = requests.get(url, auth=(username, password))
    response.raise_for_status()
    return response.content

我们可以使用上面的代码来询问服务器它的主分支是什么提交，就像这样：（这个函数非常脆弱但是很容易写出来）

In [6]:
def get_remote_master_hash(git_url, username, password):
    """Get commit hash of remote master branch, return SHA-1 hex string or
    None if no remote commits.
    """
    url = git_url + '/info/refs?service=git-receive-pack'
    response = http_request(url, username, password)
    lines = extract_lines(response)
    assert lines[0] == b'# service=git-receive-pack\n'
    assert lines[1] == b''
    if lines[2][:40] == b'0' * 40:
        return None
    master_sha1, master_ref = lines[2].split(b'\x00')[0].split()
    assert master_ref == b'refs/heads/master'
    assert len(master_sha1) == 40
    return master_sha1.decode()

### 确定没有的对象

接下来我们需要去判断服务器需要什么。toy_git 假定所有东西都存在本地（不支持 pulling）,所以我们有一个 read_tree 的函数（与 write_tree 相反）。接下来有两个函数在给定的树和提交中去递归找到对象的集合

In [7]:
def find_tree_objects(tree_sha1):
    """Return set of SHA-1 hashes of all objects in this tree
    (recursively), including the hash of the tree itself.
    """
    objects = {tree_sha1}
    for mode, path, sha1 in read_tree(sha1=tree_sha1):
        if stat.S_ISDIR(mode):
            objects.update(find_tree_objects(sha1))
        else:
            objects.add(sha1)
    return objects

def find_commit_objects(commit_sha1):
    """Return set of SHA-1 hashes of all objects in this commit
    (recursively), its tree, its parents, and the hash of the commit
    itself.
    """
    objects = {commit_sha1}
    obj_type, commit = read_object(commit_sha1)
    assert obj_type == 'commit'
    lines = commit.decode().splitlines()
    tree = next(l[5:45] for l in lines if l.startswith('tree '))
    objects.update(find_tree_objects(tree))
    parents = (l[7:47] for l in lines if l.startswith('parent '))
    for parent in parents:
        objects.update(find_commit_objects(parent))
    return objects

然后我们需要做的是，去得到那些在本地提交被引用的对象集合，减去在远程仓库被引用的对象集合。减去后的集合就是远端仓库没有的对象集合。我很肯定有更高效的办法去产生这个集合，但是对于我们这个 toy_git 来说已经不错了

In [8]:
def find_missing_objects(local_sha1, remote_sha1):
    """Return set of SHA-1 hashes of objects in local commit that are
    missing at the remote (based on the given remote commit hash).
    """
    local_objects = find_commit_objects(local_sha1)
    if remote_sha1 is None:
        return local_objects
    remote_objects = find_commit_objects(remote_sha1)
    return local_objects - remote_objects

### push！

为了执行 push 操作，我们需要发送 pkt-line 的请求去说，“将master 分支更新为此提交哈希”然后是一个包文件，其中包含上面找到的所有缺失对象的连接内容。

包文件有一个 12 字节的头（从 PACK 开始），然后每个对象用可变长度编码并使用 zlib 压缩，最后是整个包文件的 20 字节哈希。我们使用“未定义”来代表这些对象，使事情变得简单。这里有更复杂的方法缩减我们打包的文件，但是对我们来说太过分了


In [9]:
def encode_pack_object(obj):
    """Encode a single object for a pack file and return bytes
    (variable-length header followed by compressed data bytes).
    """
    obj_type, data = read_object(obj)
    type_num = ObjectType[obj_type].value
    size = len(data)
    byte = (type_num << 4) | (size & 0x0f)
    size >>= 4
    header = []
    while size:
        header.append(byte | 0x80)
        byte = size & 0x7f
        size >>= 7
    header.append(byte)
    return bytes(header) + zlib.compress(data)

def create_pack(objects):
    """Create pack file containing all objects in given given set of
    SHA-1 hashes, return data bytes of full pack file.
    """
    header = struct.pack('!4sLL', b'PACK', 2, len(objects))
    body = b''.join(encode_pack_object(o) for o in sorted(objects))
    contents = header + body
    sha1 = hashlib.sha1(contents).digest()
    data = contents + sha1
    return data

我们来到了最后一步，为了简洁我删除了一些代码

In [2]:
def push(git_url, username, password):
    """Push master branch to given git repo URL."""
    remote_sha1 = get_remote_master_hash(git_url, username, password)
    local_sha1 = get_local_master_hash()
    missing = find_missing_objects(local_sha1, remote_sha1)
    lines = ['{} {} refs/heads/master\x00 report-status'.format(
            remote_sha1 or ('0' * 40), local_sha1).encode()]
    data = build_lines_data(lines) + create_pack(missing)
    url = git_url + '/git-receive-pack'
    response = http_request(url, username, password, data=data)
    lines = extract_lines(response)
    assert lines[0] == b'unpack ok\n', \
        "expected line 1 b'unpack ok', got: {}".format(lines[0])